## Initialization

### Initializing client

To get started we need to initialize our client. The process for initializing our client is almost identical for both the local deployment and cloud deployment using Langsmith.

In [5]:
from langgraph_sdk import get_client

# If you deployed using Langsmith use this option
# Find this url on your Langsmith deployment page
example_deployed_url = (
    "https://ht-unhealthy-buffalo25-39d00f953458585aa9f7b5a4fa-g3ps4aazkq-uc.a.run.app"
)

# If you deployed locally using langgraph up -c langgraph.json use this option
# This is the default URL, and you can just call get_client() to use it
example_local_url = "http://localhost:8123"

client = get_client(url="whatever-your-url-is")

### Selecting an Assistant

To select an assistant we can search the assistants that are hosted on our client, and then select the one we want:

In [6]:
assistants = await client.assistants.search()
assistants = [a for a in assistants if not a["config"]]
assistant = assistants[0]

In our example we are only hosting a single assistant, but you have the option to host many, in which case you will most likely want to do more filtering than just selecting the first one. Each assistant is a JSON object with the following format, allowing you to select based on a variety of parameters.

In [7]:
assistant

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-06-11T20:12:45.862108+00:00',
 'updated_at': '2024-06-11T20:12:45.862108+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

### Configuring an assistant

One important thing to know is that graph can be defined to be configurable, meaning that not every instance of the graph needs to be the same (read up on [this guide](https://langchain-ai.github.io/langgraph/how-tos/configuration/) to learn more about how to create your own configurable graphs). Let's briefly show how we can configure an assistant. The first step to do is find the assistant we want to configure. In our simple example we are only hosting a single graph, so we must choose it as the graph to configure.

In [11]:
base_assistant = assistants[0]

This assitant has one configurable argument called `model`, which can take on two values: `openai` or `anthropic`. In this case, I want to create a graph that originates from this assistant that uses the `openai` option, which we can do as follows:

In [7]:
config_graph = await client.assistants.create('agent',config = {'configurable':{'model':'openai'}})

Now if we inspect our new graph, we can see that it has been configured and whenever we run it, it will select the `openai` model.

In [8]:
config_graph

{'assistant_id': '4d91047c-585c-4d1b-9ade-5b0f4d292528',
 'graph_id': 'agent',
 'created_at': '2024-06-21T23:06:08.203776+00:00',
 'updated_at': '2024-06-21T23:06:08.203776+00:00',
 'config': {'configurable': {'model': 'openai'}},
 'metadata': {}}

Being able to create configurable assistants allows you to create different graphs all based on the same underlying structure. This can be very powerful for testing different configurations of a graph or allowing users to customize their graph.

### Creating a thread

Threads are what we will actually use to run our graphs (assistants). Each thread will update the same state for the graph, meaning we can run the graph multiple times while the state will persist. We can also look back at our thread history, add meta data to different steps of our thread, and update the thread state manually if we wish. We will dive into all of those topics later in this article, but for now let’s just see how to start a thread:

In [8]:
thread = await client.threads.create()

We can examine the structure of our thread, which similar to the assistants object provides us with some information about the thread itself, including its id, timestamps, and metadata:

In [9]:
thread

{'thread_id': '6c2e8002-5712-4388-bed6-0747e9a86e31',
 'created_at': '2024-06-19T15:58:59.243657+00:00',
 'updated_at': '2024-06-19T15:58:59.243657+00:00',
 'metadata': {}}

Now we are ready to actually use our graph!

## Invoking the graph

The graph used in this example is a simple example of a StateGraph, but it allows us to show most of the API functionality. The state of our graph is defined as follows:

In [10]:
from typing import Annotated, TypedDict

from langchain_core.messages import AnyMessage

from langgraph.graph import add_messages


def update_user_info(old_info, new_info):
    if "name" not in new_info or new_info["age"] == -1:
        return old_info
    return new_info


class UserInformation(TypedDict):
    age: int
    name: str


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: Annotated[UserInformation, update_user_info]

It is important to note that our member variables can be updated by using the `Annotated` class. This is especially important when we make API calls that will update our variables. This is also a good example that your graph can hold much more information than just messages. In our example we use a very simple `UserInformation` class, but you can imagine holding much richer information in your state.

Our graph looks like follows:

<div style="text-align:center">
    <img src="./img/graph_diagram.png" style="width:30%">
</div>

The workflow is as follows: first the user inputs some message, our llm decides how to configure the call to our tool `get_user_info` , and after getting the results of the tool call we respond to our user using another LLM.

### Simple Invocation

Ok, now that we have set up our client, assistant, and thread we can actually invoke the graph above. Let’s first define the function we will use to invoke the graph, since we don’t want to have to rewrite this code every single run.

In [48]:
async def run_input(client, thread, assistant, input, metadata={}):
    # client.runs.stream will stream the results of running our graph
    async for chunk in client.runs.stream(
        thread["thread_id"],
        assistant["assistant_id"],
        input=input,
        config={"configurable": metadata},
        stream_mode="updates",
    ):
        if chunk.data and "run_id" not in chunk.data:
            print(chunk.data)

Let’s now see what happens to our graph when we run it with a simple sentence:

In [28]:
input = {
    "messages": [
        {"role": "user", "content": "Hello! My name is Bagatur and I am 26 years old."}
    ]
}

await run_input(client, thread, assistant, input)

{'llm': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_ioeUDw39bXQ2nap5f593xZMW', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-75a0d9db-df99-4b0c-b356-f24e76f5ca5a', 'example': False, 'tool_calls': [{'name': 'PersonalInfo', 'args': {'age': 26, 'name': 'Bagatur'}, 'id': 'call_ioeUDw39bXQ2nap5f593xZMW'}], 'invalid_tool_calls': [], 'usage_metadata': None}]}}
{'get_user_info': {'messages': [{'content': 'Hello! My name is Bagatur and I am 26 years old.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'abafa365-1280-439d-b2ac-d4c547284ff9', 'example': False}, {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_E6NK18NXOHFkp8CeIuF7iI3K', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}

We can see our graph ran as expected, by calling our three nodes sequentially. Let’s now examine the state to take a further look under the hood. We can get the state by using the following command: 

In [17]:
state = await client.threads.get_state(thread_id=thread["thread_id"])
state.keys()

dict_keys(['values', 'next', 'config', 'metadata', 'created_at', 'parent_config'])

Our state variable contains a variety of important information. Here is a quick summary of the keys and what they represent:

- `values` contains the actual state values, so in our case you could call `state['values']['messages']` or `state['values']['user_info']` and get the actual values of each of the state variables.
- `next`  tells us what action in the graph is next at the current state. Since we just finished running our graph and reached the end node, it is currently empty because there is no next action to take. However, if you go through the state at each point of the run you will see that the `next` value goes from `__start__` → `llm` → `get_user_info` →`respond_to_user` .
- `metadata` stores the metadata associated with our state. This is data that is outside of the agent state, but is important to keep track of across multiple runs. An example of this is shown in the next section.
- `config` tells us what the configuration of the state is. This is important for when we want to run a query starting at a previous state instead of the one we are at. An example of this is shown in the Invoking from a previous checkpoint section

### Invoking with Metadata

Let’s create a new thread to reset our state and start fresh.

In [49]:
thread = await client.threads.create()

Now let’s add some metadata to our request. In this example we are going to treat each run of our assistant as a separate “node”.  For each run, we will pass in a “node_id” as well as a “parent_node” in the metadata. This way we can easily go “back in time” and rerun our graph from a previous checkpoint. 

> NOTE: The reason we add this metadata instead of using the `parent_config` attribute is because `parent_config` tracks every individual step of a run, not the entire run itself.
>

In [50]:
input = {
    "messages": [
        {"role": "user", "content": "Hello! My name is Bagatur and I am 26 years old."}
    ]
}

metadata = {"node_id": 1, "parent_node": None}

await run_input(client, thread, assistant, input, metadata)

{'llm': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_VrA7UKg2w99BvIpsqrwjoawk', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-067609c1-4f1a-4d6d-bc1c-224a29153d37', 'example': False, 'tool_calls': [{'name': 'PersonalInfo', 'args': {'age': 26, 'name': 'Bagatur'}, 'id': 'call_VrA7UKg2w99BvIpsqrwjoawk'}], 'invalid_tool_calls': [], 'usage_metadata': None}]}}
{'get_user_info': {'messages': [{'content': 'Hello! My name is Bagatur and I am 26 years old.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '05faf88c-9f85-462e-84ee-4667de35625d', 'example': False}, {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_VrA7UKg2w99BvIpsqrwjoawk', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}

Using our `run_input` function makes it easy to pass in metadata and you can inspect the function as well as the API docs to see exactly how metadata gets passed.

We can continue our thread by creating a second node as follows:

In [51]:
input = {"messages": [{"role": "user", "content": "Hello! What is my name?"}]}
metadata = {"node_id": 2, "parent_node": 1}

await run_input(client, thread, assistant, input, metadata)

{'llm': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_b2T2e0doVrh6uB1aHLIQglYR', 'function': {'arguments': '{"age":-1,"name":"John Doe"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-808dca3a-0188-4c6b-9fc7-0037949fd820', 'example': False, 'tool_calls': [{'name': 'PersonalInfo', 'args': {'age': -1, 'name': 'John Doe'}, 'id': 'call_b2T2e0doVrh6uB1aHLIQglYR'}], 'invalid_tool_calls': [], 'usage_metadata': None}]}}
{'get_user_info': {'messages': [{'content': 'Hello! My name is Bagatur and I am 26 years old.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '05faf88c-9f85-462e-84ee-4667de35625d', 'example': False}, {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_VrA7UKg2w99BvIpsqrwjoawk', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}

Perfect! The state persisted across separate runs, and the LLM remembers the name of our user. In a future we will explore non-sequential runs, i.e. not having each run just follow the last one but choosing which checkpoint we start our run from.

### Invoking on a schedule

Running your graph on a schedule can be valuable in situations where you want to run tasks periodically. The Langgraph Cloud API allows you to do this through the `CronAssistant` class. To schedule a graph job, you need to pass a [cron expression](https://crontab.cronhub.io/) to inform the client when you want to run the graph. `Cron` jobs are run in the background and do not interfere with normal invocations of the graph.

In [ ]:
# This schedules a job to run at 15:27 (3:27PM) every day
cron_1 = await client.crons.create(assistant['assistant_id'],schedule="27 15 * * *",input={'messages':[{"role":"user","content":"What time is it?"}]})

Note that it is **very** important to delete `Cron` jobs that are no longer useful. Otherwise you could rack up unwanted API charges to the LLM! You can delete a `Cron` job using the following code:

In [ ]:
await client.crons.delete(cron_1['cron_id'])

## Querying and Updating the thread

### Getting checkpoints by metadata

Let’s say we want to start a new run from a previous state (not the current state). This state lives somewhere in our history, so we can utilize the `get_history` function to try and find it.

In [52]:
history = await client.threads.get_history(thread_id=thread["thread_id"])

This is helpful for inspecting the specifics of our current thread, but remember that the history contains all the intermediate steps a graph takes. In our case, where the graph has 5 nodes (remember that Start and End both count as nodes), our history array grows quickly. Luckily, there is a way to query by using metadata. For example if we wanted to start a run from Node 1(from the example from above) we need to find the state from the end of run with metadata node_id:1 , which we can do like so:

In [53]:
node_1_history = await client.threads.get_history(
    thread_id=thread["thread_id"], metadata={"node_id": 1}
)
# At the end of the run there will be no 'next' for the graph to execute
node_1_end_of_run = [h for h in node_1_history if h["next"] == []][0]

Now let’s explore how we could use this information to create a new branch in our thread.

### Invoking from a previous checkpoint

The following diagram describes what we would like to happen:

<div style="text-align:center">
    <img src="./img/thread_diagram.png" style="width:30%">
</div>
Basically, we want to have 3 runs of our graph, but instead of having them sequentially - we want both the second and third run to originate from the same state. We can do this by utilizing the code we used above, and passing additional metadata to our run.

In [54]:
input = {"messages": [{"role": "user", "content": "Hello! What is my age?"}]}
metadata = {
    **{"node_id": 3, "parent_node": 1},
    **{
        "thread_ts": node_1_end_of_run["checkpoint_id"],
        "thread_id": thread["thread_id"],
    },
}

await run_input(client, thread, assistant, input, metadata)

{'llm': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_yC7HUAQfcLzheepMD8SnAojR', 'function': {'arguments': '{"age":-1}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-b6a653e2-11ea-4f82-aa6a-ef321deed9ce', 'example': False, 'tool_calls': [{'name': 'PersonalInfo', 'args': {'age': -1}, 'id': 'call_yC7HUAQfcLzheepMD8SnAojR'}], 'invalid_tool_calls': [], 'usage_metadata': None}]}}
{'get_user_info': {'messages': [{'content': 'Hello! My name is Bagatur and I am 26 years old.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '05faf88c-9f85-462e-84ee-4667de35625d', 'example': False}, {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_VrA7UKg2w99BvIpsqrwjoawk', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reas

To check that everything actually worked as planned, let’s check our current state and check that message history to ensure that the message we passed to Node 2 is nowhere to be found.

In [55]:
"Hello! What is my name?" in [
    message["content"] for message in state["values"]["messages"]
]

False

Great! This has worked as expected. Being able to go back to previous states and execute new graph runs from those checkpoints is a great way to develop flexible applications that don’t require reloading or restarting everything when an error is detected or a user changes their mind.

### Updating/Patching the thread state

Lastly, let’s discuss the ability to manually change both the thread state as well as the metadata for a given state. Let’s say we incorrectly inputted data to the LLM and we want to rectify it. 

Continuing our previous example, let’s say the user mistyped their age and we want to let the graph know that without actually running it. In this case we can rectify this by using `update_state`

In [56]:
new_state = await client.threads.update_state(
    thread_id=thread["thread_id"], values={"user_info": {"name": "Bagatur", "age": 35}}
)

Let’s make sure that the state did in fact update and ask our LLM again how old we are by invoking the graph again:

In [57]:
input = {"messages": [{"role": "user", "content": "Hello! What is my age?"}]}

await run_input(client, thread, assistant, input)

{'llm': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_Ph3tYdt2UNdwqAF3kVL198Bg', 'function': {'arguments': '{"age":-1}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-373f3e22-8549-4f77-9ff2-05133099bb09', 'example': False, 'tool_calls': [{'name': 'PersonalInfo', 'args': {'age': -1}, 'id': 'call_Ph3tYdt2UNdwqAF3kVL198Bg'}], 'invalid_tool_calls': [], 'usage_metadata': None}]}}
{'get_user_info': {'messages': [{'content': 'Hello! My name is Bagatur and I am 26 years old.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '05faf88c-9f85-462e-84ee-4667de35625d', 'example': False}, {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_VrA7UKg2w99BvIpsqrwjoawk', 'function': {'arguments': '{"age":26,"name":"Bagatur"}', 'name': 'PersonalInfo'}, 'type': 'function'}]}, 'response_metadata': {'finish_reas

Voila! The LLM knows our users age updated without us having to prompt it at all.

The last thing we will talk about is patching the thread, which is used when we want to update the  metadata of a state. For example, say we actually wanted to update our last state to have `node_id:4` instead of `node_id:3`. To do this, we can call:

In [58]:
await client.threads.patch_state(thread_id=thread["thread_id"], metadata={"node_id": 4})

{'configurable': {'thread_id': '4f044e5a-6f6e-4663-923e-6333c052ce9f',
  'thread_ts': '1ef2e5d6-e39f-6d26-800e-22245f8220a7'}}

We can check that this worked by checking the metadata of our state

In [59]:
state = await client.threads.get_state(thread_id=thread["thread_id"])
print(f"Current node id is {state['metadata']['node_id']}")

Current node id is 4


Perfect! The patch worked as expected.